In [1]:
import pandas as pd
import numpy as np
from sklearn import preprocessing as pp
from sklearn import metrics
from sklearn.metrics import roc_auc_score as auc
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA

import sys
import os

In [ ]:
def saveResult(y):
    index = np.array([i+1 for i in range(len(y))])
    index.resize(len(index), 1)
    y.resize(len(index), 1)
    data = np.concatenate((index, y), -1)

    df = pd.DataFrame(data, columns=['Id', 'label'])
    df[['Id']] = df[['Id']].astype(int)

    df.to_csv('./outputs/submission1.csv', index=False)

In [2]:
def read_data(trainPath, testPath, info=True):
    #读取原始数据
    trainData = pd.read_csv(trainPath, header=None)
    testData = pd.read_csv(testPath, header=None)

    if(info):
        print('trainData shape: ', trainData.shape)
        print('testData shape: ', testData.shape)

    return trainData, testData

In [3]:

def remove_NAN(data, Thresh):
    """
    brief: 将data的每一列中非nan值少于Thresh的列删除。
    
    params:
        data: DataFrame类型。
        Thresh: int, 非nan值少于Thresh的列将被删除
    """

    data = data.dropna(axis=1, thresh=Thresh)#删除nan很多的列
    print('data shape after remove NAN columns: ', data.shape)
    
    return data

In [4]:
def remove_cols_appear_too_much(data, frequency):
    #对于那些一列中某个元素出现次数太多的，也将这些列删除
    index = []
    data3 = data.values
    data3 = pd.DataFrame(data3)
    for i in range(data3.shape[1]):
        freq = data3[i].value_counts().values[0]
        if freq>frequency:
            index.append(i)#获得重复元素超过frequency的下标
    print("indexes that will be removed:", index)
    data4 = data3.values
    data4 = np.delete(data4, index, axis=1)#删除index下标的所有列

    print("data shape after remove_cols_appear_too_much: ",data4.shape)
    return data4

In [5]:
def norm(data):
    #归一化处理
    mmScaler = pp.MinMaxScaler()
    data4 = mmScaler.fit_transform(data)
    print("data shape after normalization:", data4.shape)
    return data4

In [6]:
def process_date(data):
    """
    brief:
        处理数据中的日期，更改时间日期格式
    
    params:
        data: DataFrame类型。包含日期的列，需要被处理。
        
    return: 
        data: DataFrame类型，删除了原始data中日期所在的列
        date: DataFrame类型。将日期所在的列按照年月日时分秒映射成6个独立的属性。
    """
    dateCol = 206  #时间所在的列，手动统计的
    
    date = pd.DataFrame()
    date['date'] = pd.to_datetime(pd.Series(data[dateCol]), format='%Y-%m-%d-%H.%M.%S.%f')
    
    # 转化为6个单独的属性
    date['Year'] = date['date'].apply(lambda x: x.year)
    date['Month'] = date['date'].apply(lambda x: x.month)
    date['Day'] = date['date'].apply(lambda x: x.day)
    date['Hour'] = date['date'].apply(lambda x: x.hour)
    date['Minute'] = date['date'].apply(lambda x: x.minute)
    date['Second'] = date['date'].apply(lambda x: x.second)
    
    data = data.drop(dateCol, axis=1)
    date = date.drop('date', axis=1)
    
    return data, date

In [7]:
def process_nonNumeric(data):
    """
    brief：
        处理数据中包含的非数值类型，将其映射成one-hot形式。
        
    params：
        data: DataFrame类型。包含非数值型的列，需要被处理。
    
    return：
        data: DataFrame类型，删除了原始data中非数值的列
        newData：DataFrame类型。将非数值的列映射成one-hot的形式后产生的DataFrame类型。
    """
    nonNumericCol = [132, 135, 138, 141, 144, 147, 150, 153, 156, 159, 
                     162, 165, 168, 171, 174, 177, 180, 183, 186, 189, 
                     192, 195, 198, 201, 207, 244] #手动统计的
    
    nonNumericData = data[nonNumericCol]#提取非数值的列

    count = 0
    newData = pd.DataFrame()
    for col in nonNumericData.columns:
        #将非数值的每一列修改成one-hot的形式
        res = nonNumericData[col].value_counts()
        #count += res.index.size
        for indx in res.index:
            newData[count] = nonNumericData[col].apply(lambda x, index: 1 if x==index else 0,
                                                      args=(indx,))
            count += 1
    
    res = np.where(nonNumericData.isnull(), 1, 0)
    
    newData = np.hstack((newData.values, res))
    newData = pd.DataFrame(newData)
    #将非数值的列删除
    data = data.drop(nonNumericCol,axis=1)
    #print(count)
    return data, newData

In [16]:
trainPath = "./originalDataset/train.csv"
testPath = "./originalDataset/test.csv"
trainData, testData = read_data(trainPath, testPath)

featuresNum = trainData.shape[1]#得到列数
trainNum = trainData.shape[0]#得到行数


trainData = trainData.values
testData = testData.values

label = trainData[:,featuresNum-2].reshape((-1,1))#提取出标签
label = np.where(label==-1,0,1).reshape((-1,1)) #将标签转从{-1,1}换成{0,1}
print("label shape: ", label.shape, "sum label: ",sum(label))

#合并训练集和测试集，方便后面进行数据处理
data = np.vstack((trainData[:,0:(featuresNum-2)],testData[:,0:(featuresNum-2)]))
data = pd.DataFrame(data)
print('merge data shape: ',data.shape)

data2, nonNumericData = process_nonNumeric(data)
print('data2.shape: ', data2.shape)
print(nonNumericData.head())
data3, dateData = process_date(data2)
print('data3.shape: ', data3.shape)

#将其它列中的非数值数据转换成nan值。
#coerce参数将非数字的值转换成nan，如果是ignore参数，则不处理
data = data.apply(lambda x:pd.to_numeric(x, errors="coerce"))

trainData shape:  (112405, 248)
testData shape:  (28101, 247)
label shape:  (112405, 1) sum label:  [330]
merge data shape:  (140506, 246)
data2.shape:  (140506, 220)
   0    1    2    3    4    5    6    7    8    9   ...   908  909  910  911  \
0    0    1    0    0    0    0    1    0    0    0 ...     0    0    0    0   
1    0    0    0    0    0    0    0    0    0    0 ...     0    0    0    0   
2    0    0    0    0    0    0    0    0    0    0 ...     0    0    0    0   
3    0    0    0    1    0    0    0    1    0    0 ...     0    0    0    0   
4    0    0    0    1    0    0    0    1    0    0 ...     0    0    0    0   

   912  913  914  915  916  917  
0    0    0    0    0    0    0  
1    0    0    0    0    0    0  
2    0    0    0    0    0    0  
3    0    0    0    0    0    0  
4    0    0    0    0    0    0  

[5 rows x 918 columns]
data3.shape:  (140506, 219)


In [19]:
data5 = remove_NAN(data3, 10000)
data5.shape

data shape after remove NAN columns:  (140506, 203)


(140506, 203)

In [20]:
#合并所有的数据
data2 = np.hstack((data5.values, nonNumericData.values, dateData.values))
print('merge data, nonNumericData and dateData: ',data2.shape)

merge data, nonNumericData and dateData:  (140506, 1127)


In [22]:
data2 = pd.DataFrame(data2)
data2.head()

,0,1,2,3,4,5,6,7,8,9,...,1117,1118,1119,1120,1121,1122,1123,1124,1125,1126
0,1.28e+11,1180,1180,1190,1150,602.642,812.551,1049.8,1163.07,1163.07,...,0,0,0,0,2017,5,11,12,53,54
1,1.29e+11,1040,968,1210,NaN,298.634,706.033,928.569,NaN,NaN,...,0,0,0,0,2017,7,26,17,57,13
2,1.29e+11,1150,1070,1180,1170,217.183,530.094,945.946,1153.05,1153.05,...,0,0,0,0,2017,7,3,17,37,41
3,1.28e+11,1150,1110,1210,1190,883.341,926.378,1083.76,1164.29,1164.29,...,0,0,0,0,2017,4,30,1,43,35
4,1.28e+11,1070,1010,1140,1140,134.587,469.587,797.631,1043.61,1043.61,...,0,0,0,0,2017,4,26,17,16,40


In [23]:
data2.shape

(140506, 1127)

In [24]:
trainNum = 112405
data2 = data2.values
testRes = data2[trainNum:]
trainRes = data2[0:trainNum]
print("testRes shape:",testRes.shape)
print("trainRes shape:",trainRes.shape)

#保存测试集
version = '0506jp'
testRes = pd.DataFrame(testRes)
testRes.to_csv('./dataAfterProcess/testRes%s.csv'%(version), header=False, index=False, encoding='utf-8')

#保存训练集
trainRes1 = np.hstack((trainRes, label))
trainRes2 = pd.DataFrame(trainRes1)
trainRes2.to_csv('./dataAfterProcess/trainRes%s.csv'%(version), header=False, index=False, encoding='utf-8')

testRes shape: (28101, 1127)
trainRes shape: (112405, 1127)


In [50]:
tset

,132,135
0,EG26,EG21
1,NaN,NaN
2,NaN,NaN
3,EG30,EG20
4,EG30,EG20
5,NaN,NaN
6,EG28,EG29
7,NaN,NaN
8,EG26,EG21
9,EG28,EG29


In [22]:
#删除NAN值很多的列
Threshold = 10000
data = remove_NAN(data3, Threshold)
data

data shape after remove NAN columns:  (140506, 203)


,0,1,2,3,4,5,6,7,8,9,...,235,236,237,238,239,240,241,242,243,245
0,1.28e+11,1180,1180,1190,1150,602.642,812.551,1049.8,1163.07,1163.07,...,0.07,0,0,0,0,0,0,0,0,54
1,1.29e+11,1040,968,1210,NaN,298.634,706.033,928.569,NaN,NaN,...,0.03379,0.001,0,0,0,0,0,0,0,36
2,1.29e+11,1150,1070,1180,1170,217.183,530.094,945.946,1153.05,1153.05,...,0.04399,0,0,0,0,0,0,0,0,37
3,1.28e+11,1150,1110,1210,1190,883.341,926.378,1083.76,1164.29,1164.29,...,0.045,0,0,0,0,0,0,0,0,36
4,1.28e+11,1070,1010,1140,1140,134.587,469.587,797.631,1043.61,1043.61,...,0.03869,0,0,0,0,0,0,0,0,37
5,1.29e+11,1080,1020,1190,1210,571.101,726.371,976.971,1155.59,1155.59,...,0.03799,0,0,0,0,0,0,0,0,37
6,1.29e+11,1130,1070,1170,1230,708.328,796.015,994.349,1167.19,1167.19,...,0.03999,0,0,0,0,0,0,0,0,NaN
7,1.29e+11,1090,1130,1220,1190,173.617,441.833,857.87,1147.6,1147.6,...,0.07,0,0,0,0,0,0,0,0,54
8,1.28e+11,864,1060,1270,1210,763.209,941.871,983.102,1187.07,1187.07,...,0.045,0,0,0,0,0,0,0,0,51
9,1.29e+11,1110,1120,1110,1250,528.476,715.525,1035.43,1157.52,1157.52,...,0.07,0,0,0,0,0,0,0,0,NaN


In [23]:
data = np.hstack((data.values, nonNumericData.values, dateData.values))
data.shape


(140506, 875)

In [ ]:
def f(data):
    print(id(data))
    
data = 33
print(id(data))
f(data)
print(data)